# Train notebook - Airbus kaggle challenge

# Ship Detection- Project AIRBUS 2019
## Model Parameters
We might want to adjust these later (or do some hyperparameter optimizations)

#### Imports:

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from preprocess.pre_process_segmentation import multi_rle_encode, rle_encode, rle_decode, masks_as_image, masks_as_color, balancing_train
from preprocess.pre_process_segmentation import make_image_gen, create_aug_gen
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.utils import multi_gpu_model

import keras 
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from models.linknet50 import get_linknet50
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback

Using TensorFlow backend.


In [2]:
BATCH_SIZE = 16
EDGE_CROP = 16
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# number of validation images to use
VALID_IMG_COUNT = 900
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 8000
MAX_TRAIN_EPOCHS = 5

In [3]:
# Use of tensorflow:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()
for device in devices:
    print(device)
    
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 5123421584038050190)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4567694668810435782)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10196630034180427289)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 15560753152, 9089153444235179874)
Default GPU Device: /device:GPU:0


In [4]:
ship_dir = '../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train')# Images for training
test_image_dir = os.path.join(ship_dir, 'test')# Images for testing
model_weights_dir = "weights_models/"

label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2_clean.csv')# Images for testing
dataframe = pd.read_csv(label_dir, engine="python") # Markers for ships

In [5]:
dataframe.head(5)

,Unnamed: 0,ImageId,EncodedPixels
0,0,00003e153.jpg,NaN
1,1,0001124c7.jpg,NaN
2,2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [6]:
data_link_has_ship = balancing_train(dataframe, rate_of_has_ship=1, ship_dir_train=train_image_dir)
data_link_balanced = balancing_train(dataframe, rate_of_has_ship=0.5, ship_dir_train=train_image_dir)

print("data_link_has_ship rate:{0}, lenght: {1}".format(data_link_has_ship.EncodedPixels.notnull().sum()/len(data_link_has_ship)
                                                    ,len(data_link_has_ship)))
print("data_link_balanced rate:{0}, lenght: {1}".format(round(data_link_balanced.EncodedPixels.notnull().sum()/len(data_link_balanced),2)
                                                    ,len(data_link_balanced)))

data_link_has_ship rate:1.0, lenght: 77324
data_link_balanced rate:0.5, lenght: 154648


In [7]:
data_link_has_ship.head(5)

,ImageId,EncodedPixels
0,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
1,00021ddc3.jpg,108287 1 109054 3 109821 4 110588 5 111356 5 1...
2,00021ddc3.jpg,101361 1 102128 3 102896 4 103663 6 104430 9 1...
3,00021ddc3.jpg,74441 3 75207 5 75975 5 76743 5 77511 5 78280 ...
4,00021ddc3.jpg,74444 4 75212 4 75980 4 76748 4 77517 3 78285 ...


In [8]:
training_set, validation_set = train_test_split(data_link_has_ship, test_size=0.05)
training_set_balanced, validation_set_balanced = train_test_split(data_link_balanced, test_size=0.05)

print("length of training set", len(training_set))
print("length of validation set", len(validation_set))
print("length of unbalanced training set ", len(training_set_balanced))
print("length of unbalanced validation set", len(validation_set_balanced))

length of training set 73457
length of validation set 3867
length of unbalanced training set  146915
length of unbalanced validation set 7733


In [9]:
train_gen = make_image_gen(training_set, train_image_dir, BATCH_SIZE, (3,3))
train_x, train_y = next(train_gen)
print('x', train_x.shape, train_x.min(), train_x.max())
print('y', train_y.shape, train_x.min(), train_x.max())

x (16, 256, 256, 3) 0.0 1.0
y (16, 256, 256, 1) 0.0 1.0


### **Classification at low resolution:**

#### First pass of the classification with low resolution images dataset and Imagenet weights
- For the low resolution we use the parameter scaling = 3.
- We don't start from scratch and we use pre-trained weights from ImageNet

# Data aug

In [10]:
# from keras.preprocessing.image import ImageDataGenerator
# AUGMENT_BRIGHTNESS = False

# dg_args = dict(featurewise_center = False, 
#                   samplewise_center = False,
#                   rotation_range = 45, 
#                   width_shift_range = 0.1, 
#                   height_shift_range = 0.1, 
#                   shear_range = 0.01,
#                   zoom_range = [0.9, 1.25],  
#                   horizontal_flip = True, 
#                   vertical_flip = True,
#                   fill_mode = 'reflect',
#                    data_format = 'channels_last')
# # brightness can be problematic since it seems to change the labels differently from the images 
# if AUGMENT_BRIGHTNESS:
#     dg_args[' brightness_range'] = [0.5, 1.5]
# image_gen = ImageDataGenerator(**dg_args)

# if AUGMENT_BRIGHTNESS:
#     dg_args.pop('brightness_range')
# label_gen = ImageDataGenerator(**dg_args)

# aug_gen = create_aug_gen(train_gen, image_gen, label_gen)
# print('x', next(aug_gen)[0].shape)

# Build model

In [11]:
# Intersection over Union for Objects
def IoU(y_true, y_pred, tresh=1e-10):
    Intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    Union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (Intersection + tresh) / (Union + tresh), axis=0)
# Intersection over Union for Background
def back_IoU(y_true, y_pred):
    return IoU(1-y_true, 1-y_pred)
# Loss function
def IoU_loss(in_gt, in_pred):
    #return 2 - back_IoU(in_gt, in_pred) - IoU(in_gt, in_pred)
    return 1 - IoU(in_gt, in_pred) 

#Dice
def dice_coef(y_true, y_pred, smooth=0.001):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
def dice_loss(in_gt, in_pred):
    return 1e-3*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)
def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)

In [12]:
def get_list_files(dataframe, ship_dir_train):
    training_set, validation_set = train_test_split(dataframe, test_size=0.05)
    
    training_set_has_ship = balancing_train(training_set, rate_of_has_ship=1, ship_dir_train=ship_dir_train)
    validation_set_has_ship = balancing_train(validation_set, rate_of_has_ship=1, ship_dir_train=ship_dir_train)
    training_set_balanced = balancing_train(training_set, rate_of_has_ship=0.5, ship_dir_train=ship_dir_train)
    validation_set_balanced = balancing_train(validation_set, rate_of_has_ship=0.5, ship_dir_train=ship_dir_train)

    dict_dataset = {'training_set_has_ship': training_set_has_ship,
                   'validation_set_has_ship': validation_set_has_ship,
                   'training_set_balanced': training_set_balanced,
                   'validation_set_balanced': validation_set_balanced
                   }
    
    return dict_dataset

In [13]:
def callbacks_list(weight_path, scaling):
    checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min', save_weights_only=False)

    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                       patience=2, verbose=1, mode='min',
                                       min_delta=0.001, cooldown=1, min_lr=1e-7)

    early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=8)
    tensorboard =  TensorBoard(log_dir="../logs/log{0}".format(str(scaling)), update_freq='batch')
    return [checkpoint, reduceLROnPlat, early]

In [14]:
def get_image_generator(training_set, validation_set, scaling, train_image_dir, batch_size):
    step_count_train = min(MAX_TRAIN_STEPS, training_set.shape[0]//batch_size)
    training_gen = make_image_gen(training_set, train_image_dir, batch_size, (scaling, scaling))
    
    step_count_valid = validation_set.shape[0]//batch_size
    validation_gen = make_image_gen(validation_set, train_image_dir, batch_size, (scaling, scaling))
    
    return step_count_train, training_gen, step_count_valid, validation_gen

In [15]:
def save_model(model, nbr_gpu=1, scalling=1, print_details=False):
    # Sauvegarde des poids
    if nbr_gpu > 1:
        # save weights multi-gpu
        weights_path_clf_scal = model_weights_dir + "model_seg_scal{0}_{1}_weights.best.h5".format(str(scaling), "p_gpu")
        if print_details:
            print("Save weights to :", weights_path_clf_scal)
        model.save_weights(weights_path_clf_scal)  # save multi-gpu model weights
            
        # save weights single-gpu
        s_gpu_model = model.layers[-2]   #get single GPU model weights
        if print_details:
            print("Save weights to :", weights_path_clf_scal)
        weights_path_clf_scal = model_weights_dir + "model_seg_scal{0}_{1}_weights.best.h5".format(str(scaling), "s_gpu")
        s_gpu_model.save_weights(weights_path_clf_scal)  # save single-gpu model weights
    else:
        weights_path_clf_scal = model_weights_dir + "model_seg_scal{0}_{1}_weights.best.h5".format(str(scaling), "s_gpu")
        if print_details:
            print("Save weights to :", weights_path_clf_scal)
        model.save_weights(weights_path_clf_scal)  # save single-gpu model weights

In [16]:
def segmentation_training(dict_dataset, train_image_dir, weights=None, scaling=1, batch_size=BATCH_SIZE,
                            nbr_gpu=0, print_details=False, epochs=MAX_TRAIN_EPOCHS, nbr_cpu=1):
    train_gen = make_image_gen(dict_dataset['training_set_has_ship'], train_image_dir, batch_size, (scaling, scaling))
    train_x, train_y = next(train_gen)
    
    if print_details:
        print('x', train_x.shape, train_x.min(), train_x.max())
        print('y', train_y.shape, train_x.min(), train_x.max())

    # Definition of the model with the input shape
    seg_model = get_linknet50(input_shape=train_x.shape[1:])
    
    # Load weights
    if weights is not None:
        if print_details:
            print("Load weights...")
        seg_model.load_weights(weights, by_name=True)

    weight_path = model_weights_dir + "model_clf_checkpoint_scal{0}_weights.hdf5".format(str(scaling))
    callbacks = callbacks_list(weight_path, scaling)
   
    # Model appliqué sur un jeux équilibré
    if print_details:
        print("Get image generator for balance dataset...")
    step_count_train, training_gen, step_count_valid, validation_gen = get_image_generator(dict_dataset['training_set_has_ship'],
                                                                                           dict_dataset['validation_set_has_ship'],
                                                                                           scaling, 
                                                                                           train_image_dir,
                                                                                           batch_size)
    if print_details:
        print("step_count_train =", step_count_train)
        print("step_count_valid =", step_count_valid)
    
    if nbr_gpu > 1:
        model = multi_gpu_model(seg_model, gpus=nbr_gpu)
    else:
        model = seg_model
    
    if print_details:
        seg_model.summary()
        
    model.compile(optimizer=Adam(), loss=dice_loss, 
                  metrics=[IoU, back_IoU, dice_coef, 'binary_accuracy', true_positive_rate])
    
    if print_details:
        print("Start model...")
    loss_history1 = [model.fit_generator(training_gen,
                                     steps_per_epoch=step_count_train,
                                     epochs=epochs,
                                     callbacks=callbacks,
                                     validation_data=validation_gen,
                                     validation_steps=step_count_valid,
                                     use_multiprocessing=nbr_cpu>1,
                                     workers=nbr_cpu)]
    
    print("Model Evaluation Balanced Data: ", model.evaluate_generator(validation_gen,
                                              step_count_valid,
                                              workers=1,
                                              verbose=1))
    
    # save weights
    save_model(model, nbr_gpu=nbr_gpu, scalling=scaling, print_details=print_details)
    
    
    # Model appliqué sur un jeux non équilibré
#     if print_details:
#         print("Get image generator for balance dataset...")
#     step_count_train, training_gen, step_count_valid, validation_gen = get_image_generator(dict_dataset['training_set_balanced'],
#                                                                                            dict_dataset['validation_set_balanced'],
#                                                                                            scaling, 
#                                                                                            train_image_dir,
#                                                                                            batch_size)
    
#     if print_details:
#         print("step_count_train =", step_count_train)
#         print("step_count_valid =", step_count_valid)
    
#     if print_details:
#         print("Start model...")
        
#     loss_history2 = [model.fit_generator(training_gen,
#                                  steps_per_epoch=step_count_train,
#                                  epochs=epochs//2,
#                                  callbacks=callbacks,
#                                  validation_data=validation_gen,
#                                  validation_steps=step_count_valid,
#                                  use_multiprocessing=nbr_cpu>1,
#                                  workers=nbr_cpu)]
    
#     print("Model Evaluation Unbalanced Data: ", model.evaluate_generator(validation_gen,
#                                           step_count_valid,
#                                           workers=1,
#                                           verbose=1))
    
#     # save weights
#     save_model(model, nbr_gpu=nbr_gpu, scalling=scaling, print_details=print_details)
            


In [17]:
dict_dataset = get_list_files(dataframe, train_image_dir)

In [ ]:
weights_path2 = model_weights_dir + "model_clf_scal{0}_{1}_weights.best.h5".format(str(2), "s_gpu")
segmentation_training(dict_dataset, train_image_dir, weights=weights_path2, 
                        scaling=2, batch_size=8, nbr_gpu=1, print_details=True, 
                        epochs=MAX_TRAIN_EPOCHS, nbr_cpu=1)

x (8, 384, 384, 3) 0.0 1.0
y (8, 384, 384, 1) 0.0 1.0


/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Load weights...
Get image generator for balance dataset...
step_count_train = 8000
step_count_valid = 484
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 390, 390, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 192, 192, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 192, 192, 64) 256         conv1[0][0]                      
___

Start model...
Epoch 1/5
 696/8000 [=>............................] - ETA: 37:03 - loss: -0.3798 - IoU: 0.1901 - back_IoU: 0.4914 - dice_coef: 0.3801 - binary_accuracy: 0.9714 - true_positive_rate: 0.6792

In [ ]:
weights_path2 = model_weights_dir + "model_seg_scal{0}_{1}_weights.best.h5".format(str(2), "s_gpu")
segmentation_training(dict_dataset, train_image_dir, weights=weights_path2, 
                        scaling=1, batch_size=4, nbr_gpu=1, print_details=True, 
                        epochs=MAX_TRAIN_EPOCHS, nbr_cpu=1)

In [ ]:
#run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

model.compile(optimizer=Adam(), loss=dice_loss, 
              metrics=[IoU, back_IoU, dice_coef, 'binary_accuracy', true_positive_rate])

step_count_train = min(MAX_TRAIN_STEPS, training_set.shape[0]//BATCH_SIZE)
training_gen = make_image_gen(training_set, train_image_dir, BATCH_SIZE, IMG_SCALING)
training_aug_gen =  create_aug_gen(training_gen, image_gen, label_gen)

step_count_valid = validation_set.shape[0]//BATCH_SIZE
validation_gen = make_image_gen(validation_set, train_image_dir, BATCH_SIZE, IMG_SCALING)

print("step_count_train =", step_count_train)
print("step_count_valid =", step_count_valid)


loss_history = [model.fit_generator(training_aug_gen,
                                 steps_per_epoch=step_count_train,
                                 epochs=MAX_TRAIN_EPOCHS,
                                 callbacks=callbacks_list,
                                 validation_data=validation_gen,
                                 validation_steps=step_count_valid,
                                workers=1 # the generator is not very thread safe
                                           )]

In [ ]:
def show_loss(loss_history):
    epochs = np.concatenate([mh.epoch for mh in loss_history])
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 15))
    
    _ = ax1.plot(epochs, np.concatenate([mh.history['loss'] for mh in loss_history]), 'b-',
                 epochs, np.concatenate([mh.history['val_loss'] for mh in loss_history]), 'r-')
    ax1.legend(['Training', 'Validation'])
    ax1.set_title('Loss')
    
    _ = ax2.plot(epochs, np.concatenate([mh.history['IoU'] for mh in loss_history]), 'b-',
                 epochs, np.concatenate([mh.history['val_IoU'] for mh in loss_history]), 'r-')
    ax2.legend(['Training', 'Validation'])
    ax2.set_title('IoU accuracy (%)')
    
    _ = ax3.plot(epochs, np.concatenate([mh.history['back_IoU'] for mh in loss_history]), 'b-',
                 epochs, np.concatenate([mh.history['val_back_IoU'] for mh in loss_history]), 'r-')
    ax3.legend(['Training', 'Validation'])
    ax3.set_title('back_IoU accuracy (%)')

show_loss(loss_history)

In [ ]:
pred_y = model.predict(next(validation_gen)[0])
print(pred_y.shape, pred_y.min(axis=0).max(), pred_y.max(axis=0).min(), pred_y.mean())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 6))
ax.hist(pred_y.ravel(), np.linspace(0, 1, 20))
ax.set_xlim(0, 1)
ax.set_yscale('log', nonposy='clip')

## Prepare Full Resolution Model
Here we account for the scaling so everything can happen in the model itself

In [ ]:

if IMG_SCALING is not None:
    fullres_model = models.Sequential()
    fullres_model.add(layers.AvgPool2D(IMG_SCALING, input_shape = (None, None, 3)))
    fullres_model.add(model_1)
    fullres_model.add(layers.UpSampling2D(IMG_SCALING))
else:
    fullres_model = model_1
fullres_model.save('fullres_model_1.h5')

# Visualisation of predictions

In [ ]:
from matplotlib.cm import get_cmap
def raw_prediction(img, path=test_image_dir):
    c_img = imread(os.path.join(path, img))
    c_img = np.expand_dims(c_img, 0)/255.0
    cur_seg = fullres_model.predict(c_img)[0]
    return cur_seg, c_img[0]

def smooth(cur_seg):
    return binary_opening(cur_seg>0.99, np.expand_dims(disk(2), -1))

def predict(img, path=test_image_dir):
    cur_seg, c_img = raw_prediction(img, path=path)
    return smooth(cur_seg), c_img

## Get a sample of each group of ship count
samples = valid_df.groupby('ships').apply(lambda x: x.sample(1))
fig, m_axs = plt.subplots(samples.shape[0], 4, figsize = (15, samples.shape[0]*4))
[c_ax.axis('off') for c_ax in m_axs.flatten()]

for (ax1, ax2, ax3, ax4), c_img_name in zip(m_axs, samples.ImageId.values):
    first_seg, first_img = raw_prediction(c_img_name, train_image_dir)
    ax1.imshow(first_img)
    ax1.set_title('Image: ' + c_img_name)
    ax2.imshow(first_seg[:, :, 0], cmap=get_cmap('jet'))
    ax2.set_title('Model Prediction')
    reencoded = masks_as_color(multi_rle_encode(smooth(first_seg)[:, :, 0]))
    ax3.imshow(reencoded)
    ax3.set_title('Prediction Masks')
    ground_truth = masks_as_color(masks.query('ImageId=="{}"'.format(c_img_name))['EncodedPixels'])
    ax4.imshow(ground_truth)
    ax4.set_title('Ground Truth')
    
fig.savefig('validation.png')